In [1]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly_express as px
import seaborn as sns
import umap

/home/lippincm/miniconda3/envs/Interstellar/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/lippincm/miniconda3/envs/Interstellar/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/lippincm/miniconda3/envs/Interstellar/lib/pyth

In [2]:
nELISA_plate_430420_SH_SY5Y_path = pathlib.Path(
    "../../Data/clean/Plate2/nELISA_plate_430420_SH_SY5Y.csv"
)
manual_cluster_1_path = pathlib.Path(
    "../../Data/clean/Plate2/Manual_Treatment_Clusters_1.csv"
)

manual_cluster_2_path = pathlib.Path(
    "../../Data/clean/Plate2/Manual_Treatment_Clusters_2.csv"
)


nELISA_plate_430420_SH_SY5Y = pd.read_csv(nELISA_plate_430420_SH_SY5Y_path)
manual_clusters_1 = pd.read_csv(manual_cluster_1_path)
manual_clusters_2 = pd.read_csv(manual_cluster_2_path)

nELISA_orgingal_plate = nELISA_plate_430420_SH_SY5Y.copy()

In [3]:
# select data only columns and make floats
nELISA_data_values = nELISA_orgingal_plate.filter(like="NSU", axis=1)
nELISA_data_values = nELISA_data_values.astype("float")
nELISA_data_values.head()

,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],...,TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
0,0.050305,-0.046947,1.361487,0.553065,3.161210,0.323546,0.200321,-4.628914,-1.455375,1.411887,...,-1.053944,1.931219,0.439120,-1.538831,30.157265,-0.733615,0.413442,-0.897618,-1.841281,0.048389
1,-0.136531,1.014299,-1.829310,-2.860160,3.326872,-0.293770,0.520861,1.004449,-0.906373,2.202077,...,-0.871663,3.274879,-0.738697,0.816301,29.581649,-0.939235,-0.385612,-1.410167,-1.406394,-0.863555
2,1.240569,0.229261,1.995245,-2.499106,-4.232099,-1.807823,-0.656196,0.746356,-0.001466,-3.299824,...,-0.251360,1.923855,-0.523146,0.204165,31.313849,-2.824671,1.220794,-1.488981,1.496881,-0.107094
3,0.629592,1.492948,-1.857380,-0.308989,-0.640024,0.713347,-0.554004,-1.260493,0.067077,1.431232,...,-1.318833,2.165008,-0.282784,-0.779901,28.934441,-0.304395,0.177668,-0.300231,0.083451,-1.798261
4,-0.375887,0.390727,-0.026048,-0.563376,1.802794,0.231624,1.355083,-0.230052,-0.209977,0.476215,...,-1.003948,1.439554,0.407104,-0.753367,32.680499,-1.739692,-1.222531,0.841332,-0.464893,0.077351


In [4]:
# normalize data via max value in each column
max_values = nELISA_data_values.max()  # find max value in each column
nELISA_data_values_sensor_max_norm = nELISA_data_values.div(
    max_values
)  # divide each value in each column by max value in that column
nELISA_data_values_sensor_max_norm.head()

,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],...,TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
0,0.030033,-0.030537,0.275591,0.307953,0.561603,0.134832,0.057937,-1.275958,-0.929072,0.292741,...,-0.488091,0.179877,0.165775,-0.775352,0.730801,-0.156319,0.120292,-0.465569,-0.599550,0.048768
1,-0.081511,0.659766,-0.370288,-1.592570,0.591034,-0.122423,0.150645,0.276876,-0.578604,0.456579,...,-0.403676,0.305027,-0.278871,0.411300,0.716852,-0.200133,-0.112195,-0.731414,-0.457944,-0.870331
2,0.740639,0.149126,0.403876,-1.391531,-0.751851,-0.753377,-0.189787,0.205733,-0.000936,-0.684186,...,-0.116407,0.179191,-0.197497,0.102870,0.758829,-0.601883,0.355194,-0.772292,0.487408,-0.107934
3,0.375876,0.971110,-0.375970,-0.172049,-0.113703,0.297274,-0.160230,-0.347454,0.042820,0.296752,...,-0.610764,0.201652,-0.106756,-0.392959,0.701168,-0.064861,0.051693,-0.155721,0.027173,-1.812370
4,-0.224410,0.254154,-0.005273,-0.313694,0.320274,0.096525,0.391920,-0.063414,-0.134044,0.098739,...,-0.464938,0.134082,0.153689,-0.379590,0.791947,-0.370695,-0.355700,0.436375,-0.151376,0.077958


In [5]:
# print mean and stdev of first data column before and after normalization to check normalization
print(f"NSU nELISA mean of Activin A: {nELISA_data_values['Activin A [NSU]'].mean()}")
print(f"NSU nELISA STDEV of Activin A: {nELISA_data_values['Activin A [NSU]'].std()}")

print(
    f"NSU sensor normalized nELISA mean of Activin A: {nELISA_data_values_sensor_max_norm['Activin A [NSU]'].mean()}"
)
print(
    f"NSU sensor normalized nELISA STDEV of Activin A: {nELISA_data_values_sensor_max_norm['Activin A [NSU]'].std()}"
)

NSU nELISA mean of Activin A: -0.281989801113817
NSU nELISA STDEV of Activin A: 0.7676688394550193
NSU sensor normalized nELISA mean of Activin A: -0.16835212877937208
NSU sensor normalized nELISA STDEV of Activin A: 0.4583097786138692


In [6]:
# display max columns pandas
pd.set_option("display.max_columns", None)
nELISA_orgingal_plate.head()

,plate_name,plate_barcode,position_x,cell_type,incubation inducer,inhibitor,inhibitor_concentration_value,inhibitor_concentration_unit,inhibitor_concentration,inducer1,inducer1_concentration_value,inducer1_concentration_unit,inducer1_concentration,inducer2,inducer2_concentration_value,inducer2_concentration_unit,inducer2_concentration,plate_position,user_plate_id,user_sample_id,position_y,nelisa_sample_dilution_fold,nelisa_sample_comments,nelisa_sample_pass_qc,nelisa_sample_id,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],BMP4 [NSU],BMP6 [NSU],BMP7 [NSU],BMP9 [NSU],C5/C5a [NSU],Calbindin [NSU],CCL1 [NSU],CCL11 [NSU],CCL13 [NSU],CCL15 [NSU],CCL16 [NSU],CCL17 [NSU],CCL18 [NSU],CCL19 [NSU],CCL2 [NSU],CCL20 [NSU],CCL21 [NSU],CCL22 [NSU],CCL23 [NSU],CCL24 [NSU],CCL25 [NSU],CCL27 [NSU],CCL28 [NSU],CCL3 [NSU],CCL4 [NSU],CCL5 [NSU],CCL7 [NSU],CCL8 [NSU],CD14 [NSU],CD163 [NSU],CD276 (B7-H3) [NSU],CD27L [NSU],CD30 [NSU],CD40L [NSU],CNTF [NSU],CRP [NSU],CX3CL1 [NSU],CXCL1 [NSU],CXCL10 [NSU],CXCL11 [NSU],CXCL12 (alpha) [NSU],CXCL12 (beta) [NSU],CXCL13 [NSU],CXCL14 [NSU],CXCL16 [NSU],CXCL17 [NSU],CXCL3 [NSU],CXCL4 [NSU],CXCL5 [NSU],CXCL6 [NSU],CXCL7 [NSU],CXCL9 [NSU],Cytochrome C [NSU],EGF [NSU],EGFR [NSU],EMMPRIN [NSU],FAS-L [NSU],FGF-1 [NSU],FGF-19 [NSU],FGF-2 [NSU],FGF-21 [NSU],FGF-4 [NSU],FGF-6 [NSU],FGF-7 (KGF) [NSU],FGF-9 [NSU],FGFR3 (IIIc) [NSU],FLRG (FSTL3) [NSU],Flt-3 Ligand [NSU],G-CSF [NSU],GDF-11 (BMP-11) [NSU],GDF-15 (MIC-1) [NSU],GDNF [NSU],GM-CSF [NSU],Granzyme B [NSU],Growth Hormone (Somatotropin) [NSU],HGF [NSU],HVEM [NSU],ICAM-1 [NSU],ICAM-2 [NSU],IFN alpha 2 (alpha 2b) [NSU],IFN beta [NSU],IFN gamma [NSU],IFN-epsilon [NSU],IGF-1 [NSU],IL-1 alpha [NSU],IL-1 beta [NSU],IL-1 R1 [NSU],IL-1 RA/RN [NSU],IL-10 [NSU],IL-11 [NSU],IL-12 p35 [NSU],IL-12 p40 [NSU],IL-12 p70 [NSU],IL-15 [NSU],IL-15/IL-15R alpha complex [NSU],IL-16 [NSU],IL-17A [NSU],IL-17B [NSU],IL-17C [NSU],IL-17D [NSU],IL-17E (IL-25) [NSU],IL-17F [NSU],IL-18 [NSU],IL-2 [NSU],IL-2 RA [NSU],IL-21 [NSU],IL-22 [NSU],IL-22 BP [NSU],IL-23 [NSU],IL-24 [NSU],IL-27 [NSU],IL-28A [NSU],IL-29 [NSU],IL-3 [NSU],IL-31 [NSU],IL-32 (alpha) [NSU],IL-33 [NSU],IL-35 [NSU],IL-4 [NSU],IL-5 [NSU],IL-6 [NSU],IL-6 R alpha [NSU],IL-7 [NSU],IL-8 [NSU],IL-9 [NSU],Leptin [NSU],LIF [NSU],LOX1 (OLR1) [NSU],M-CSF [NSU],M-CSF R (CD115) [NSU],Mesothelin [NSU],MIF [NSU],MMP-1 [NSU],MMP-10 [NSU],MMP-12 [NSU],MMP-2 [NSU],MMP-3 [NSU],MMP-7 [NSU],MMP-9 [NSU],NF-L [NSU],NGF beta [NSU],NRG1 beta 1 [NSU],Oncostatin M (OSM) [NSU],Osteopontin (OPN) [NSU],PCSK9 [NSU],PDGF-BB [NSU],PLGF [NSU],PTX3 (Pentraxin 3) [NSU],Resistin [NSU],SAA [NSU],SCF [NSU],ST2 (IL-33R) [NSU],TGF-beta 1 (LAP domain in precursor) [NSU],TGF-beta 1 (total) [NSU],TGF-beta 2 [NSU],TGF-beta 3 [NSU],Tie-2 [NSU],TIMP1 [NSU],Tissue Factor (TF) [NSU],TNF alpha [NSU],TNF RI [NSU],TNF RII [NSU],TNF RIII (Lymphotoxin Beta R) [NSU],TPO (Thrombopoietin) [NSU],TRAIL [NSU],TREM2 [NSU],TSLP [NSU],TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU],Activin A [pgML],AITRL (GITR Ligand) [pgML],Amphiregulin [pgML],Amyloid beta [pgML],APRIL [pgML],BAFF [pgML],BCMA (TNFRSF17) [pgML],BDNF [pgML],BMP2 [pgML],BMP3 [pgML],BMP4 [pgML],BMP6 [pgML],BMP7 [pgML],BMP9 [pgML],C5/C5a [pgML],Calbindin [pgML],CCL1 [pgML],CCL11 [pgML],CCL13 [pgML],CCL15 [pgML],CCL16 [pgML],CCL17 [pgML],CCL18 [pgML],CCL19 [pgML],CCL2 [pgML],CCL20 [pgML],CCL21 [pgML],CCL22 [pgML],CCL23 [pgML],CCL24 [pgML],CCL25 [pgML],CCL27 [pgML],CCL28 [pgML],CCL3 [pgML],CCL4 [pgML],CCL5 [pgML],CCL7 [pgML],CCL8 [pgML],CD14 [pgML],CD163 [pgML],CD276 (B7-H3) [pgML],CD27L [pgML],CD30 [pgML],CD40L [pgML],CNTF [pgML],CRP [pgML],CX3CL1 [pgML],CXCL1 [pgML],CXCL10 [pgML],CXCL11 [pgML],CXCL12 (alpha) [pgML],CXCL12 (beta) [pgML],CXCL13 [pgML],CXCL14 [pgML],CXCL16 [pgML],CXCL17 [pgML],CXCL3

In [7]:
# rename columns to remove special character "/"
nELISA_orgingal_plate.columns = nELISA_orgingal_plate.columns.str.replace("/", "_")

# set umap parameters
umap_params = umap.UMAP(
    n_neighbors=6,
    min_dist=0.8,
    n_components=2,
    metric="cosine",
    spread=1.1,
    init="random",
    random_state=0,
)

# fit and transform data for umap
proj_2d = umap_params.fit_transform(nELISA_data_values_sensor_max_norm)

# add umap coordinates to dataframe of metadata and raw data
nELISA_orgingal_plate["umap_1"] = proj_2d[:, 0]
nELISA_orgingal_plate["umap_2"] = proj_2d[:, 1]

# add manual clusters columns to dataframe
nELISA_plate_430420 = pd.merge(
    nELISA_orgingal_plate,
    manual_clusters_2,
    on=(
        "inducer1",
        "inducer1_concentration_value",
        "inhibitor",
        "inhibitor_concentration_value",
        "inducer2",
        "inducer2_concentration_value",
    ),
    how="inner",
)

nELISA_plate_430420["inducer1_plus_concentration"] = (
    nELISA_plate_430420["inducer1"]
    + "_"
    + nELISA_plate_430420["inducer1_concentration"]
)

nELISA_plate_430420["inducer1_plus_inhibitor"] = (
    nELISA_plate_430420["inducer1"] + "_" + nELISA_plate_430420["inhibitor"]
)

nELISA_plate_430420["inducer1_plus_concentration_plus_inhibitor"] = (
    nELISA_plate_430420["inducer1"]
    + "_"
    + nELISA_plate_430420["inducer1_concentration"]
    + "_"
    + nELISA_plate_430420["inhibitor"]
)

# define output paths
nELISA_plate_430420_out_path = pathlib.Path(
    "./results/nELISA_plate_430420_umap_SH-SY5Y.csv"
)
# write to csv
nELISA_plate_430420.to_csv(nELISA_plate_430420_out_path, index=False)